In [1]:
import ee
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import time

# Path to your service account key file
SERVICE_ACCOUNT_FILE = 'automate2-428111-c5d352d7461e.json'  # Update the path if needed

# Initialize Earth Engine with service account credentials
credentials = ee.ServiceAccountCredentials('automate2acc@automate2-428111.iam.gserviceaccount.com', SERVICE_ACCOUNT_FILE)
ee.Initialize(credentials)

# Authentication for Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive']
drive_credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=drive_credentials)

print("Authentication successful")

# Test: List the first 10 image collections
collections = ee.ImageCollection('COPERNICUS/S1_GRD').limit(10)
collection_list = collections.getInfo()

for item in collection_list['features']:
    print(item['id'])


Authentication successful
COPERNICUS/S1_GRD/S1A_EW_GRDH_1SDH_20141003T003636_20141003T003740_002658_002F54_ECFA
COPERNICUS/S1_GRD/S1A_EW_GRDH_1SDH_20141003T003740_20141003T003840_002658_002F54_A006
COPERNICUS/S1_GRD/S1A_EW_GRDH_1SDH_20141003T003840_20141003T003920_002658_002F54_4DD1
COPERNICUS/S1_GRD/S1A_EW_GRDH_1SDH_20141003T035403_20141003T035507_002660_002F63_5346
COPERNICUS/S1_GRD/S1A_EW_GRDH_1SDH_20141003T035507_20141003T035607_002660_002F63_DBA5
COPERNICUS/S1_GRD/S1A_EW_GRDH_1SDH_20141003T035607_20141003T035707_002660_002F63_9B6F
COPERNICUS/S1_GRD/S1A_EW_GRDH_1SDH_20141003T035707_20141003T035807_002660_002F63_ADCF
COPERNICUS/S1_GRD/S1A_EW_GRDH_1SDH_20141003T035807_20141003T035907_002660_002F63_AEE6
COPERNICUS/S1_GRD/S1A_EW_GRDH_1SDH_20141003T035907_20141003T035940_002660_002F63_DA91
COPERNICUS/S1_GRD/S1A_EW_GRDH_1SDH_20141003T070457_20141003T070602_002662_002F6A_3888


In [2]:
df = pd.read_csv('saved_attribute_table.csv') 

In [3]:
df

,fid,event,last_alive,first_stan,last_stand,first_gap,notes,confirmed,centroid_x,centroid_y,area
0,22,standing&b/u,NaN,2016-06,2017-12,2018-06,NaN,checked,556651.360,9533727.893,105.412
1,23,standing&b/u,NaN,2016-06,2017-06,2017-12,NaN,checked,555897.816,9533599.821,156.257
2,46,standing&b/u,2016-06,2016-12,2023-04,2023-10,very few branches and mostly gap now,checked,561176.381,9523245.382,106.495
3,62,standing&b/u,2018-06,2018-12,2019-06,2020-06,NaN,checked,563785.032,9527067.637,207.746
4,66,b/u,2023-09,NaN,NaN,2023-10,NaN,checked,563329.777,9527728.359,265.294
...,...,...,...,...,...,...,...,...,...,...,...
102,936,?b/u,2022-10,NaN,NaN,2022-11,NaN,?,557663.011,9544779.686,41.274
103,938,b/u,2022-10,NaN,NaN,2022-11,NaN,checked,557568.600,9544912.789,122.074
104,954,standing&b/u,2019-06,2019-12,2020-06,2020-11,NaN,checked,558675.979,9548226.623,78.080
105,958,b/u,2022-03,NaN,NaN,2022-06,NaN,checked,559284.896,9548405.319,85.460


In [2]:
# Function to create a folder
def create_folder(name):
    file_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    file = service.files().create(body=file_metadata, fields='id').execute()
    return file.get('id')

# Ensure all folders are created
def ensure_folders(num_folders):
    folder_ids = {}
    for index in range(1, num_folders + 1):
        folder_name = f'gap{index}'
        folder_id = create_folder(folder_name)
        folder_ids[folder_name] = folder_id
        print(f'Created folder {folder_name} with ID: {folder_id}')
    return folder_ids

# Create 107 folders
folder_ids = ensure_folders(107)
print("Folder creation completed.")


Created folder gap1 with ID: 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Created folder gap2 with ID: 1o6AVi9S9IzBAVqEOoAO79_AiagQkkdeH
Created folder gap3 with ID: 1yBz2lsRxPOei-vRZP9iF-9iwDbhMVMdN
Created folder gap4 with ID: 1tREcD9sneA0nGlIWAYp73k0MjDr4W1N7
Created folder gap5 with ID: 1hvZXlqoNpi9qlzKCrtwjhKnyQk7BrJ6O
Created folder gap6 with ID: 1ryBtOhjig_6xQ5KPrkdoO8Tqb3fixjRa
Created folder gap7 with ID: 1OmsmnoNa9aUExeD2PIUnti5_W5PDrq1h
Created folder gap8 with ID: 1PTdtUqPvsS4YNDZtnN2bgTLHU4DaUUpF
Created folder gap9 with ID: 1b0kFvPVtmbes7T2KGaIvfIchFu81gvao
Created folder gap10 with ID: 1cvHLxehWNn4w_X4Mr0HgAk5EF_5f54o6
Created folder gap11 with ID: 128t8yhDcerv-6rAXdoQQVV1lmSc_y_bH
Created folder gap12 with ID: 13qwKFrcp1xegxnOEXgv_-XF57EpFBlM-
Created folder gap13 with ID: 14xLR2yD_z6M5wnWZj1e8X76aAmZ_D25x
Created folder gap14 with ID: 1vV6v0Sk8L_TtIKZceJo3xmXkLozdUon_
Created folder gap15 with ID: 1lq1STgbxzSLyMvDx366RKZj0zMEYyYCw
Created folder gap16 with ID: 1wWbw1B5ugxAGA0Be5E

In [6]:
def export_sentinel1_images(start_date, interval_days, easting, northing, export_region, folder_name, iteration):
    point = ee.Geometry.Point([easting, northing], 'EPSG:32720')  # Assuming UTM zone 20S
    sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(point) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .select(['VV', 'VH'])

    start_date_obj = ee.Date(start_date)
    today = ee.Date(datetime.now().strftime('%Y-%m-%d'))
    num_images = today.difference(start_date_obj, 'day').divide(interval_days).ceil().toInt()

    for i in range(num_images.getInfo()):
        current_start_date = start_date_obj.advance(i * interval_days, 'day')
        end_date = current_start_date.advance(interval_days, 'day')

        sentinel1_subset = sentinel1.filterDate(current_start_date, end_date).first()

        # Check if the image is valid
        if sentinel1_subset.getInfo() is None:
            print(f"No image found for date range: {current_start_date.format('YYYY-MM-dd').getInfo()} to {end_date.format('YYYY-MM-dd').getInfo()}")
            continue

        image_date = sentinel1_subset.date().format('YYYY-MM-dd').getInfo()  # Get the actual date of the image

        # Ensure the description is valid
        export_description = f'sentinelexport_gap{iteration}_{current_start_date.format("YYYYMMDD").getInfo()}'

        task = ee.batch.Export.image.toDrive(
            image=sentinel1_subset.select('VV'),
            description=export_description,
            folder=folder_name,
            scale=10,
            region=export_region,
            fileFormat='GeoTIFF',
            formatOptions={'cloudOptimized': True}
        )
        task.start()

        print(f'Exporting image taken on: {image_date} in folder {folder_name}')

In [11]:
# Process the first two rows for testing
for index, row in df.head(2).iterrows():
    easting = row['centroid_x']
    northing = row['centroid_y']
    start_date = row['first_gap']
    interval_days = 80  # Adjust to avoid generating too many files
    folder_name = f'gap{index + 1}'  # Create a unique folder name for each gap

    if folder_name in folder_ids:
        folder_id = folder_ids[folder_name]
        # Define the export region around the point (buffered by 1000 meters and then bounds)
        point = ee.Geometry.Point([easting, northing], 'EPSG:32720')
        export_region = point.buffer(500).bounds()  # Adjust buffer size as needed

        # Debug print statements to ensure correct iteration
        print(f'Starting export for gap {index + 1} with folder name {folder_name}')
        export_sentinel1_images(start_date, interval_days, easting, northing, export_region, folder_id, index + 1)
        print(f'Completed export for gap {index + 1}')
    else:
        print(f'Folder {folder_name} not found.')

print("Export tasks for the first two rows started.")

Starting export for gap 1 with folder name gap1
Exporting image taken on: 2018-06-04 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2018-09-08 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2018-11-19 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2019-01-30 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2019-04-24 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2019-07-17 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2019-09-27 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2019-12-20 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2020-03-13 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2020-05-24 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2020-08-16 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2020-11-08 in folder 1_N3CXtstWV0w2

In [10]:
from datetime import datetime

In [19]:
from googleapiclient.discovery import build
from google.oauth2 import service_account

# Path to your service account key file
SERVICE_ACCOUNT_FILE = 'automate2-428111-c5d352d7461e.json'

# Authentication for Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive']
drive_credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=drive_credentials)

# Function to list contents of a folder
def list_folder_contents(folder_id):
    query = f"'{folder_id}' in parents and trashed=false"
    results = service.files().list(q=query, spaces='drive', fields='nextPageToken, files(id, name)').execute()
    items = results.get('files', [])

    if not items:
        print(f'No files found in folder with ID: {folder_id}')
    else:
        print(f'Files in folder with ID: {folder_id}')
        for item in items:
            print(f"{item['name']} ({item['id']})")

# Folder IDs
folder_id_gap1 = '1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv'
folder_id_gap2 = '1o6AVi9S9IzBAVqEOoAO79_AiagQkkdeH'

# List contents of gap1 and gap2 folders
print("Contents of gap1 folder:")
list_folder_contents(folder_id_gap1)
print("\nContents of gap2 folder:")
list_folder_contents(folder_id_gap2)


Contents of gap1 folder:
No files found in folder with ID: 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv

Contents of gap2 folder:
No files found in folder with ID: 1o6AVi9S9IzBAVqEOoAO79_AiagQkkdeH


In [1]:
def list_all_files():
    results = service.files().list(spaces='drive', fields='nextPageToken, files(id, name)').execute()
    items = results.get('files', [])

    if not items:
        print('No files found.')
    else:
        print('Files:')
        for item in items:
            print(f"{item['name']} ({item['id']})")

# List all files in the service account's Google Drive
list_all_files()

NameError: name 'service' is not defined

In [18]:
import ee
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime

# Path to your service account key file
SERVICE_ACCOUNT_FILE = 'automate2-428111-c5d352d7461e.json'

# Initialize Earth Engine with service account credentials
credentials = ee.ServiceAccountCredentials('automate2acc@automate2-428111.iam.gserviceaccount.com', SERVICE_ACCOUNT_FILE)
ee.Initialize(credentials)

# Authentication for Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive']
drive_credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=drive_credentials)

print("Authentication successful")

# Load the dataframe  # Update the path to your CSV file

# Pre-existing folder IDs
folder_ids = {
    'gap1': '1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv',
    'gap2': '1o6AVi9S9IzBAVqEOoAO79_AiagQkkdeH',
    # Add the rest of your folder IDs here
}

# Function to export Sentinel-1 images
def export_sentinel1_images(start_date, interval_days, easting, northing, export_region, folder_id, iteration):
    point = ee.Geometry.Point([easting, northing], 'EPSG:32720')  # Assuming UTM zone 20S
    sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(point) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .select(['VV', 'VH'])

    start_date_obj = ee.Date(start_date)
    today = ee.Date(datetime.now().strftime('%Y-%m-%d'))
    num_images = today.difference(start_date_obj, 'day').divide(interval_days).ceil().toInt()

    for i in range(num_images.getInfo()):
        current_start_date = start_date_obj.advance(i * interval_days, 'day')
        end_date = current_start_date.advance(interval_days, 'day')

        sentinel1_subset = sentinel1.filterDate(current_start_date, end_date).first()

        # Check if the image is valid
        if sentinel1_subset.getInfo() is None:
            print(f"No image found for date range: {current_start_date.format('YYYY-MM-dd').getInfo()} to {end_date.format('YYYY-MM-dd').getInfo()}")
            continue

        image_date = sentinel1_subset.date().format('YYYY-MM-dd').getInfo()  # Get the actual date of the image

        # Ensure the description is valid
        export_description = f'sentinelexport_gap{iteration}_{current_start_date.format("YYYYMMDD").getInfo()}'

        task = ee.batch.Export.image.toDrive(
            image=sentinel1_subset.select('VV'),
            description=export_description,
            folder=folder_id,
            scale=10,
            region=export_region,
            fileFormat='GeoTIFF',
            formatOptions={'cloudOptimized': True}
        )
        task.start()

        print(f'Exporting image taken on: {image_date} in folder {folder_id}')

# Process the first two rows for testing
for index, row in df.head(2).iterrows():
    easting = row['centroid_x']
    northing = row['centroid_y']
    start_date = row['first_gap']
    interval_days = 80  # Adjust to avoid generating too many files
    folder_name = f'gap{index + 1}'  # Create a unique folder name for each gap

    if folder_name in folder_ids:
        folder_id = folder_ids[folder_name]
        # Define the export region around the point (buffered by 1000 meters and then bounds)
        point = ee.Geometry.Point([easting, northing], 'EPSG:32720')
        export_region = point.buffer(500).bounds()  # Adjust buffer size as needed

        # Debug print statements to ensure correct iteration
        print(f'Starting export for gap {index + 1} with folder name {folder_name}')
        export_sentinel1_images(start_date, interval_days, easting, northing, export_region, folder_id, index + 1)
        print(f'Completed export for gap {index + 1}')
    else:
        print(f'Folder {folder_name} not found.')

print("Export tasks for the first two rows started.")


Authentication successful
Starting export for gap 1 with folder name gap1
Exporting image taken on: 2018-06-04 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2018-09-08 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2018-11-19 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2019-01-30 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2019-04-24 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2019-07-17 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2019-09-27 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2019-12-20 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2020-03-13 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2020-05-24 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2020-08-16 in folder 1_N3CXtstWV0w2Ffuj6KK0YzjFs9w-XZv
Exporting image taken on: 2020-11-0